In [3]:
import pandas as pd
import numpy as np
import json

In [160]:
# Getting data from orginal CVS requested in Proposal
import requests
import numpy as np
df = pd.DataFrame()
def get_company_data_from_csv():
    cols_to_use = [0,1,2,3,4,5,6,7]
    csv_file_path = "../../assets/companylist.csv"
    df = pd.read_csv(csv_file_path, 
#         nrows=101, 
        usecols= cols_to_use,
        encoding='utf-8'
        )
    df = df.fillna('Empty')
    df['IPOyear'] = df['IPOyear'].astype(str)
    df['IPOyear'] = df['IPOyear'].str.strip('.0')
    df = df.rename(index=str, columns = {"Symbol": "SYM", "Name": "CO.", "LastSale": "LP", "MarketCap": "MRKCAP", "IPOyear": "YR", "Sector": "SEC", "industry": "IND", "Summary Quote": "SUMQTE"})
    df['SYM'] = df['SYM'].str.strip()
    df['SEC'] = df['SEC'].str.strip()
    df['SEC_CONV'] = np.where(df['SEC']=="Consumer Durables", 'Consumer Industry', df['SEC'])
    df['SEC_CONV'] = np.where(df['SEC']=="Consumer Non-Durables", 'Consumer Industry', df['SEC'])
    df['SEC_CONV'] = np.where(df['SEC']=='Consumer Services', 'Consumer Industry', df['SEC'])
    df["SEC"] = df["SEC"].replace("Empty", "No Industry Identified")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Empty", "No Industry Identified")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Consumer Durables", "Consumer Industry")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Consumer Non-Durables", "Consumer Industry")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Technology", "Technology/Energy")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Energy", "Technology/Energy")
    df = df.SEC_CONV.unique()
    df = json.dumps(df.tolist())
    return df
get_company_data_from_csv()

'["Health Care", "Finance", "Consumer Industry", "Technology/Energy", "No Industry Identified", "Capital Goods", "Miscellaneous", "Public Utilities", "Basic Industries", "Transportation"]'

In [156]:
# Getting distinct listing of sectors
import requests
import numpy as np
df = pd.DataFrame()
def get_distinct_sector_listing():
    cols_to_use = [0,1,2,3,4,5,6,7]
    csv_file_path = "../../assets/companylist.csv"
    df = pd.read_csv(csv_file_path, 
#         nrows=101, 
        usecols= cols_to_use,
        encoding='utf-8'
        )
    df = df.fillna('Empty')
    df['IPOyear'] = df['IPOyear'].astype(str)
    df['IPOyear'] = df['IPOyear'].str.strip('.0')
    df = df.rename(index=str, columns = {"Symbol": "SYM", "Name": "CO.", "LastSale": "LP", "MarketCap": "MRKCAP", "IPOyear": "YR", "Sector": "SEC", "industry": "IND", "Summary Quote": "SUMQTE"})
    df['SYM'] = df['SYM'].str.strip()
    df['SEC'] = df['SEC'].str.strip()
    df['SEC_CONV'] = np.where(df['SEC']=="Consumer Durables", 'Consumer Industry', df['SEC'])
    df['SEC_CONV'] = np.where(df['SEC']=="Consumer Non-Durables", 'Consumer Industry', df['SEC'])
    df['SEC_CONV'] = np.where(df['SEC']=='Consumer Services', 'Consumer Industry', df['SEC'])
    df["SEC"] = df["SEC"].replace("Empty", "No Industry Identified")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Empty", "No Industry Identified")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Consumer Durables", "Consumer Industry")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Consumer Non-Durables", "Consumer Industry")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Technology", "Technology/Energy")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Energy", "Technology/Energy")
    section_conversion = df.SEC_CONV.unique()
    print(section_conversion)
    get_distinct_industries = df.IND.unique()
    return section_conversion
get_distinct_sector_listing()

['Health Care' 'Finance' 'Consumer Industry' 'Technology/Energy'
 'No Industry Identified' 'Capital Goods' 'Miscellaneous'
 'Public Utilities' 'Basic Industries' 'Transportation']


array(['Health Care', 'Finance', 'Consumer Industry', 'Technology/Energy',
       'No Industry Identified', 'Capital Goods', 'Miscellaneous',
       'Public Utilities', 'Basic Industries', 'Transportation'],
      dtype=object)

In [42]:
# Getting list of 100 industries from 10 sectors. 
import requests
import pandas as pd
import numpy as np
df = pd.DataFrame()
def get_industries_by_sector():
    cols_to_use = [0,1,2,3,4,5,6,7]
    csv_file_path = "../../assets/companylist.csv"
    df = pd.read_csv(csv_file_path, 
#         nrows=101, 
        usecols= cols_to_use,
        encoding='utf-8'
        )
#     df = df.fillna('Empty')
    df['IPOyear'] = df['IPOyear'].astype(str)
    df['IPOyear'] = df['IPOyear'].str.strip('.0')
    df = df.rename(index=str, columns = {"Symbol": "SYM", "Name": "CO.", "LastSale": "LP", "MarketCap": "MRKCAP", "IPOyear": "YR", "Sector": "SEC", "industry": "IND", "Summary Quote": "SUMQTE"})
    df['SYM'] = df['SYM'].str.strip()
    df['SEC'] = df['SEC'].str.strip()
    df['SEC_CONV'] = np.where(df['SEC']=="Consumer Durables", 'Consumer Industry', df['SEC'])
    df['SEC_CONV'] = np.where(df['SEC']=="Consumer Non-Durables", 'Consumer Industry', df['SEC'])
    df['SEC_CONV'] = np.where(df['SEC']=='Consumer Services', 'Consumer Industry', df['SEC'])
    df["SEC"] = df["SEC"].replace("Empty", "No Industry Identified")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Empty", "No Industry Identified")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Consumer Durables", "Consumer Industry")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Consumer Non-Durables", "Consumer Industry")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Technology", "Technology/Energy")
    df["SEC_CONV"] = df["SEC_CONV"].replace("Energy", "Technology/Energy")
    df = df.groupby('SEC_CONV').head(100)
    df = df.iloc[:, [8,3]]
#     df['MRKCAP'] = df['MRKCAP'].str.replace(',', '')
    df['MRKCAP'] = df['MRKCAP'].str.replace('$', '')
    df['MRKCAP'] = df['MRKCAP'].astype(int)
    df['MRKCAP'].replace(r'[KM]+$', '', regex=True).astype(float) * \
    df['MRKCAP'].str.extract(r'[\d\.]+([KM]+)', expand=False)
#     df['MRKCAP'].fillna(1)
#     df.replace(['K','M'], [10**3, 10**6]).astype(int))
#     df['MRKCAP'] = (df.MRKCAP.replace(r'[KMB]+$', '', regex=True).astype(float) * df.MRKCAP.str.extract(r'[\d\.]+([KMB]+)', expand=False).fillna(1).replace(['K','M', 'B'], [10**3, 10**6, 10**9]).astype(int))
    return df
get_industries_by_sector()

ValueError: invalid literal for int() with base 10: '733.08M'

In [23]:
#API Data Pull
import pandas as pd
import requests
# symbol = "YI"
def get_stock_data_from_alphavantage_by_symbol(symbol):
    Alpha_Advantage_API = "https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords=" + symbol + "&apikey=GJS1X1O8CLA8KA4H"
    resp = requests.get(Alpha_Advantage_API)
    if resp.status_code == 200:
        print(resp.status_code)
        data = resp.json()
        data = data['bestMatches']
        df2 = pd.DataFrame.from_dict(data, orient='columns')
        df2.columns = ['SYM', 'CO.', 'Type', 'Country', 'LP-Time', 'LP-End-Time', 'UTC-Type', 'Dollar-Type', 'Last-List-Price']
        df2['SYM'] = df2['SYM'].str.strip()
        df2.loc[df2['SYM'] == str(symbol)]
        return df2
    else:
        print("API did not pull any data from alphavantage")
# Executes the function
get_stock_data_from_alphavantage_by_symbol(symbol)


200


,SYM,CO.,Type,Country,LP-Time,LP-End-Time,UTC-Type,Dollar-Type,Last-List-Price
0,YI,111 Inc.,Equity,United States,09:30,16:00,UTC-04,USD,1.0000
1,YINN,Direxion Daily FTSE China Bull 3X Shares,ETF,United States,09:30,16:00,UTC-04,USD,0.8000
2,YYY,YieldShares High Income ETF,ETF,United States,09:30,16:00,UTC-04,USD,0.6667
3,BOSQF,The Yield Growth Corp.,Equity,United States,09:30,16:00,UTC-04,USD,0.6667
4,YRD,Yirendai Ltd.,Equity,United States,09:30,16:00,UTC-04,USD,0.4000
5,YIN,Yintech Investment Holdings Limited,Equity,United States,09:30,16:00,UTC-04,USD,0.4000
6,YTEN,Yield10 Bioscience Inc.,Equity,United States,09:30,16:00,UTC-04,USD,0.3333
7,YGEHY,Yingli Green Energy Holding Company Limited,Equity,United States,09:30,16:00,UTC-04,USD,0.2857
8,YIPI,Yippy Inc.,Equity,United States,09:30,16:00,UTC-04,USD,0.1667


In [116]:
#Merging data that is found within the Second Dataframe from the API
def merge_company_data_for_single_api(sym1, sym2):
#     sym1 = get_stock_data_from_alphavantage_by_symbol()
#     sym2 = get_company_data_from_csv()
    return sym1.merge(sym2)

200


,SYM,CO.,Type,Country,LP-Time,LP-End-Time,UTC-Type,Dollar-Type,Last-List-Price,LP,MRKCAP,YR,SEC,IND,SUMQTE
0,YIN,Yintech Investment Holdings Limited,Equity,United States,09:30,16:00,UTC-04,USD,0.4000,4.95,$353.66M,2016,Finance,Investment Bankers/Brokers/Service,https://www.nasdaq.com/symbol/yin


In [178]:
#Info from API
import pandas as pd
import requests
def get_sector_stats():
    Alpha_Advantage_API_Sector = "https://www.alphavantage.co/query?function=SECTOR&apikey=GJS1X1O8CLA8KA4H"
    resp = requests.get(Alpha_Advantage_API_Sector)
    if resp.status_code == 200:
        print(resp.status_code)
        data = resp.json()
        df3 = pd.DataFrame.from_dict(data, orient='columns')
        df3.columns = ['SEC', 'Real_Time', '1_Day_Performance', '5_Day_Performance', '1_Month_Performance', '3_Month_Performance', 'YTD_Performance', '1_Year_Performance', '3_Year_Performance', '5_Year_Performance', '10_Year_Performance']
#         df3['SEC'] = df3['SEC'].str.strip()
#         df2.loc[df2['SYM'] == str(symbol)]
        df3 = df3['SEC']
        df3 = df3.replace('NaN', '')
        return df3
    else:
        print("API did not pull any data from alphavantage")
get_sector_stats()

200


Communication Services                                              NaN
Consumer Discretionary                                              NaN
Consumer Staples                                                    NaN
Energy                                                              NaN
Financials                                                          NaN
Health Care                                                         NaN
Industrials                                                         NaN
Information               US Sector Performance (realtime & historical)
Information Technology                                              NaN
Last Refreshed                                   04:20 PM ET 05/23/2019
Materials                                                           NaN
Real Estate                                                         NaN
Utilities                                                           NaN
Name: SEC, dtype: object

In [60]:
#Articles via API
import pandas as pd
import requests
def get_stock_news():
    stock_news_api = "https://stocknewsapi.com/api/v1?tickers=YI&items=50&type=article&token=qqygybi5zike7wfmu21lawdnfndqhhi53ndszk83"
    resp = requests.get(stock_news_api)
    if resp.status_code == 200:
        print(resp.status_code)
        data = resp.json()
        data = data['data']
        df4 = pd.DataFrame.from_dict(data, orient='columns')
        df4.columns = ['DATE', 'IMG_URL', 'NEWS_URL', 'Sentiment', 'Source_Name', 'Tags', 'Text', 'SYM', 'Title', 'Type']
        return df4
    else:
        print("API did not pull any data from alphavantage")
get_stock_news()

200


,DATE,IMG_URL,NEWS_URL,Sentiment,Source_Name,Tags,Text,SYM,Title,Type
0,"Thu, 16 May 2019 19:11:29 -0400",https://stocknewsapi.com/images/v1/g/4/transcr...,https://seekingalpha.com/article/4264778-111-i...,Neutral,Seeking Alpha,[CallScript],"111, Inc's (YI) CEO Junling Liu on Q1 2019 Res...",[YI],"111, Inc's (YI) CEO Junling Liu on Q1 2019 Res...",Article


In [72]:
#Web Scrap
import pandas as pd
import requests
from bs4 import BeautifulSoup
symbols = ['yi','bhf']
companyimages = []
nasdaq_website = "https://www.nasdaq.com/symbol/bhf"
page = requests.get(nasdaq_website)
soup = BeautifulSoup(page.text, 'html.parser')
review_text_elem = soup.find_all(class_='quote-symbol-icon')

for item in review_text_elem:
    companyimages.append(item)

companyimages
# review_text_elem



[<img alt="BHF" class="quote-symbol-icon" height="40" src="https://www.nasdaq.com/logos/BHF.gif" width="100"/>]